In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline

In [2]:
def logit(z): return 1./(1.+np.exp(-z))

In [3]:
def generate_players(num_of_players, skill_cap):
    ## returns a list of player's skill
    return [round(random.uniform(0,skill_cap),2) for i in range(num_of_players)]


def generate_games(players, num_of_games, scale=0.3, style='pystan'):

    if style == 'pystan':
        
        player1=[]
        player2=[]
        outcome=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )

            ##pystan player id is ONE based
            player1.append(p1+1)
            player2.append(p2+1)
            
            outcome.append(*random.choices([1,0],weights=[win_rate,1-win_rate]))

        return np.array([player1,player2,outcome])
    
    elif style == 'pygm':
        games=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )
            
            games.append((p1,p2,*random.choices([1,-1],weights=[win_rate,1-win_rate])))
        return games
    
    assert False 


In [4]:
def prediction_accuracy(skill_data, samples, true_players, num_valid_game=1000, scale=0.3):
    
    valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(num_valid_game):
        i,j,result=valid_games[0][g],valid_games[1][g],valid_games[2][g]

        prediction = logit( skill_data['scale']*(samples['skill'][:,i-1]-samples['skill'][:,j-1]) ).mean()

        acc+= int( (prediction >= 0.5 and result==1) or (prediction<0.5 and result==0) )
    return acc/num_valid_game

def prediction_accuracy_2(skill_data, samples, valid_data, scale=0.3):
    
    #valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(valid_data.shape[1]):
        p1,p2,result=valid_data[0][g],valid_data[1][g],valid_data[2][g]

        win_rate = logit( skill_data['scale']*(samples['skill'][:,p1-1]-samples['skill'][:,p2-1]) ).mean()
        
        predict_result = random.choices([1,0],weights=[win_rate,1-win_rate])

        if predict_result == result:
            acc += 1
    return acc/valid_data.shape[1]

In [5]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(30,10); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  We'll save a version of it so we don't have to recompile every time.

In [6]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model1.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model1.pkl', 'wb') as f: pickle.dump(sm, f)

We also need the observed data: number of players and games, which pairs played each game, and who won:

Now, we can perform MCMC on the model, and extract the samples:

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [7]:
def run_data(nplayer):
    scale=0.5
    num_of_players=nplayer
    result={}
    #Generate games 
    print("start generating")
    Data = []
    for trials in range(10):
        p = generate_players(num_of_players,62)
        games = generate_games(p,3000, scale, style='pystan')#####
        games_pool = games[:,:2500]
        valid_games = games[:,2500:]
        Data.append((p,games_pool,valid_games))


    for num_of_games in [3,5,10,20,35,50,70,100,500,700,1000,2500]:
        starting_num_of_games=time.time()
        trials_result=[]
        print("starting num_of_games =",num_of_games)
        for trials in range(10):
            true_players = Data[trials][0]

            train_games = Data[trials][1][:,:num_of_games]
            print("true skills:",true_players)
            skill_data = {
                'N': num_of_players,
                'E': num_of_games,
                'scale': scale,###
                'win':train_games[2],
                'PA': train_games[0],
                'PB': train_games[1]
            }
            starting=time.time()
            fit = sm.sampling(data=skill_data, iter=10000, chains=4)####
            print("spent:",round(time.time()-starting,3))
            samples = fit.extract()
            print("predicted skills:",samples['skill'].mean(0))
            acc=prediction_accuracy_2(skill_data,samples,Data[trials][2])
            print("acc:",acc,"\n")
            trials_result.append(acc)

        result[num_of_games]=trials_result
        print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
    return result


In [ ]:
result1 = run_data(10)

start generating
starting num_of_games = 3
true skills: [39.99, 10.12, 59.86, 3.94, 14.14, 45.74, 22.15, 43.87, 12.63, 10.16]
spent: 1.89
predicted skills: [ 0.04185143 -7.86284175 -0.13053837 -5.61783235 -0.06985272 -0.06428981
  5.40689963  7.98882077 -0.01564971 -0.05898119]
acc: 0.57 

true skills: [3.5, 24.99, 19.48, 36.22, 0.59, 16.64, 25.65, 37.65, 58.7, 11.87]
spent: 1.894
predicted skills: [-5.53850785  8.00911405 -7.87727008  0.01074088 -0.02317081 -0.04538135
  0.07568578  0.02319807 -0.09127522  5.40536397]
acc: 0.55 

true skills: [50.59, 61.89, 30.54, 44.36, 17.51, 61.65, 33.88, 13.7, 27.87, 1.95]
spent: 2.081
predicted skills: [ 5.4568015  -0.0967995   7.84678539  0.15261692 -7.92176031 -0.049557
 -0.04252496 -5.37502616 -0.04114176  0.04534043]
acc: 0.594 

true skills: [40.85, 24.84, 61.88, 44.77, 61.18, 57.08, 18.2, 53.29, 55.34, 1.76]
spent: 1.867
predicted skills: [-5.51025181e+00  3.34267776e-03  5.45638787e+00  4.14562085e+00
  3.10754059e-03 -5.74836944e-02 -8.34

spent: 2.233
predicted skills: [ -1.19350798  -8.24083592  11.05495994  -8.84386715   3.94577794
   3.93143528   5.10812432   6.71081025   0.74285708 -12.88024899]
acc: 0.712 

true skills: [3.5, 24.99, 19.48, 36.22, 0.59, 16.64, 25.65, 37.65, 58.7, 11.87]
spent: 2.356
predicted skills: [ -4.22020965   3.25073311 -10.35713142   9.51117832 -12.66613071
  -8.90629833  -0.46829539  11.69859562   9.57419672   2.56766278]
acc: 0.724 

true skills: [50.59, 61.89, 30.54, 44.36, 17.51, 61.65, 33.88, 13.7, 27.87, 1.95]
spent: 2.257
predicted skills: [  9.23393145  11.71002506  -1.41891766  -5.05688385 -12.98666577
   9.63277617   5.24282906  -1.97419224  -4.87945256  -9.71095297]
acc: 0.754 

true skills: [40.85, 24.84, 61.88, 44.77, 61.18, 57.08, 18.2, 53.29, 55.34, 1.76]
spent: 2.327
predicted skills: [ -3.14764835 -10.48911683   6.08728203   1.93686449   7.5328138
   1.87810632  -7.40489653   4.74483149  12.89665095 -14.72254761]
acc: 0.81 

true skills: [0.82, 50.41, 40.46, 48.23, 49.76, 24

spent: 3.099
predicted skills: [ 10.48223478  11.25171984  -1.31778094   4.3397423   -8.04668818
  15.56230051   2.36932031 -12.62648529  -3.95296508 -18.29039135]
acc: 0.92 

true skills: [40.85, 24.84, 61.88, 44.77, 61.18, 57.08, 18.2, 53.29, 55.34, 1.76]
spent: 3.012
predicted skills: [ -2.07966556 -14.12041216  17.09384517  -5.15644177  12.25855368
   5.67733853  -8.82687424   3.40025903   7.85621528 -15.9835588 ]
acc: 0.886 

true skills: [0.82, 50.41, 40.46, 48.23, 49.76, 24.54, 3.85, 59.48, 58.29, 2.01]
spent: 3.004
predicted skills: [-17.53886856  10.76842267  -2.16366808   2.68002338   2.2129478
  -5.05009358  -4.08114668  12.48855139  12.4045014  -11.40547753]
acc: 0.87 

true skills: [61.12, 31.27, 2.94, 33.2, 50.47, 28.05, 25.63, 23.5, 55.49, 25.05]
spent: 3.379
predicted skills: [ 15.93850891   2.11772914 -15.74547895   4.00878051   7.31157119
  -0.95583866  -6.95319033  -7.29024037  10.56768207  -8.33853964]
acc: 0.872 

true skills: [32.6, 11.57, 55.03, 50.17, 19.48, 23.

spent: 24.364
predicted skills: [-17.21754576   6.73068098   1.14605733   4.972837     7.06187617
  -5.57801933 -13.8509512   16.52968958  14.36331322 -14.83622959]
acc: 0.914 

true skills: [61.12, 31.27, 2.94, 33.2, 50.47, 28.05, 25.63, 23.5, 55.49, 25.05]
spent: 24.984
predicted skills: [ 22.10345175  -0.65933271 -19.62622662   1.59790509   9.02720249
  -3.87360568  -5.95918391  -7.94823075  12.35043511  -6.48882506]
acc: 0.906 

true skills: [32.6, 11.57, 55.03, 50.17, 19.48, 23.22, 43.41, 61.36, 43.91, 12.5]
spent: 23.571
predicted skills: [ -3.24869289 -18.52436389  19.0268152   12.07735516 -12.34532107
  -9.67519753   3.14509085  23.60833123   4.64247852 -17.56081021]
acc: 0.954 

true skills: [20.12, 39.77, 31.01, 24.43, 31.75, 60.0, 57.84, 17.54, 7.46, 14.84]


In [ ]:
def plot_result(d):
    
    M = np.array([d[k] for k in d])
    x = np.array([k for k in d])
    y = np.mean(M, axis = 1)
    ystd = scipy.stats.sem(M, axis = 1)
    print(ystd)
    
    plt.figure()
    plt.scatter(x, y, color = 'black')
    plt.errorbar(x, y, yerr = ystd, color = 'black', capsize = 5)
    plt.show()
    
plot_result(result)